In [1]:
import sys; sys.path.insert(0, '..')
from database.dataset import *

from feature_extraction.feature_extraction import CandidatesFeatureExtraction, CandidatesFeatureExtractionMP
from mc_candidate_proposal.hough_mc import HoughCalcificationDetection
from general_utils.plots import plot_bboxes_over_image, simple_im_show, simple_im_show2
from metrics.metrics import get_tp_fp_fn, quick_circle_comparison, circle_comparison

db = INBreast_Dataset(
        return_lesions_mask=True,
        level='image',
        max_lesion_diam_mm=1.0,
        extract_patches=True,
        extract_patches_method='all',  # 'centered'
        patch_size=256,
        stride=256,
        min_breast_fraction_roi=0.5,
        normalize=None,
        n_jobs=-1,
)

In [2]:
dehazing_params = {'omega': 0.9, 'window_size': 11, 'radius': 40, 'eps': 1e-5}

hough1_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                 'param1': 300, 'param2': 8,  'minRadius': 2, 'maxRadius': 20}

hough2_params = {'method': cv2.HOUGH_GRADIENT, 'dp': 1, 'minDist': 20,
                 'param1': 300, 'param2': 10,  'minRadius': 2, 'maxRadius': 20}

back_ext_radius = 50

erosion_iter = 20
erosion_size = 5


hd = HoughCalcificationDetection(dehazing_params, back_ext_radius,
                                 Path.cwd().parent / 'data/hough_img',    # "/media/vzalevskyi/DATA/ProjectsData/calc-det/no_dehaizing",
                                 hough1_params, hough2_params,
                                 erosion_iter=erosion_iter,
                                 erosion_size=erosion_size)


In [3]:
idx = 0
db_sample = db[idx]
image = db_sample['img']
image_id = db.df.iloc[idx].img_id
image_mask = db_sample['lesion_mask']
true_bboxes = db_sample['lesion_bboxes']
lesion_rads = db_sample['radiuses']

In [4]:
h1_circles = hd.detect(
    image, image_id, load_processed_images=True, hough2=False)[0]

In [5]:
h1_circles_img = plot_bboxes_over_image(image, [h1_circles], [(0, 255, 0)], ['circ'])
print(h1_circles.shape)

(1490, 3)


In [6]:
TP, FP, FN = circle_comparison(h1_circles, image_mask)
print({'image_id':image_id, 'TP': TP, 'FP': FP, 'FN': FN})

{'image_id': 20586908, 'TP': 27, 'FP': 1457, 'FN': 10}


In [7]:
h1_circles[:,2] = h1_circles[:,2]/np.sqrt(2)

In [10]:
tp, fp, fn, gt_predicted, close_fp = get_tp_fp_fn(lesion_bboxes=true_bboxes, radiuses=lesion_rads, detections=h1_circles,
             max_dist=20, min_iou=0.0, exact=False, lesion_mask=image_mask)


In [11]:
len(fn)

33